In [2]:
import pyensembl
from pyensembl import Genome
import pandas as pd
import csv
import pymysql
import operator
import time
import datetime
from termcolor import colored
import math

#### Import reference GTF to retrieve genes names by coordinates

In [5]:
# wget ftp://ftp.ensembl.org/pub/release-98/gtf/homo_sapiens/Homo_sapiens.GRCh38.98.gtf.gz
data = Genome(reference_name='GRCh38', annotation_name='98', gtf_path_or_url='REF/Homo_sapiens.GRCh38.98.gtf')

In [3]:
# data.index()

In [6]:
data.gene_names_at_locus(contig=1, position=630755, end=634922, strand='+') 

['MTATP6P1', 'MTATP8P1', 'MTCO1P12', 'MTCO2P12', 'MTCO3P12']

### Processing transcript expression table from R

In [46]:
df_trans_expr = pd.read_csv('BALLGOWN_RES/bg_transcripts_expression.txt', sep='\t', skiprows=(0), header=(0))
df_trans_expr.head()

,t_id,chr,strand,start,end,t_name,num_exons,length,gene_id,gene_name,...,cov.BG12,FPKM.BG12,cov.BG13,FPKM.BG13,cov.BG14,FPKM.BG14,cov.BG15,FPKM.BG15,cov.BG16,FPKM.BG16
11,11,1,-,14409,192299,MSTRG.4.8,3,6817,MSTRG.4,.,...,14.542374,1.442821,13.637791,1.766196,21.559107,2.694932,20.800241,2.111696,12.477324,1.154332
47,47,1,-,185217,195411,ENST00000623083.4,10,1397,ENSG00000279457.4,WASH9P,...,19.456787,1.930404,26.013676,3.368966,10.763598,1.345472,31.347738,3.182506,0.000000,0.000000
49,49,1,-,187891,187958,ENST00000612080.1,1,68,ENSG00000273874.1,MIR6859-2,...,14.262753,1.415078,15.576286,2.017246,27.847542,3.480999,24.015854,2.438154,19.812614,1.832952
105,105,1,+,629062,629433,ENST00000416931.1,1,372,ENSG00000225972.1,MTND1P23,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
106,106,1,+,629084,630536,MSTRG.3.1,1,1453,MSTRG.3,.,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [103]:
len(df_trans_expr)

17306

####  A) Retrieve all possible names from PYENSEMBL

In [47]:
# TRANSCRIPTS
# Query pyensembl to fill missing gene names (if exists)
# Store list of loci that have more than one gene name for querying UCSC
ambiguous = []
i = 0
j = 0
for index, row in df_trans_expr.iterrows():
    if row['gene_name'] == '.':
        name = data.gene_names_at_locus(strand=row['strand'], contig=row['chr'], position=row['start'], end=row['end'])
        if len(name) == 1:
            i+=1
            df_trans_expr.at[index, 'gene_name'] = name[0]
        elif len(name) > 1:
            k=0
            for token in name:
                if '.' in token:
                    k+=1
            if k == len(name) -1:
                for token in name:
                    if '.' not in token:
                        i+=1
                        df_trans_expr.at[index, 'gene_name'] = token
            else:
                j+=1
                ambiguous.append([row['gene_id'], name, row['strand'], row['chr'], row['start'], row['end']])
print(f'Filled {i} unique gene names. Appended {j} loci with multiple gene names to list.')

Filled 5201 unique gene names. Appended 1998 loci with multiple gene names to list.


In [48]:
df_trans_expr.head()

,t_id,chr,strand,start,end,t_name,num_exons,length,gene_id,gene_name,...,cov.BG12,FPKM.BG12,cov.BG13,FPKM.BG13,cov.BG14,FPKM.BG14,cov.BG15,FPKM.BG15,cov.BG16,FPKM.BG16
11,11,1,-,14409,192299,MSTRG.4.8,3,6817,MSTRG.4,.,...,14.542374,1.442821,13.637791,1.766196,21.559107,2.694932,20.800241,2.111696,12.477324,1.154332
47,47,1,-,185217,195411,ENST00000623083.4,10,1397,ENSG00000279457.4,WASH9P,...,19.456787,1.930404,26.013676,3.368966,10.763598,1.345472,31.347738,3.182506,0.000000,0.000000
49,49,1,-,187891,187958,ENST00000612080.1,1,68,ENSG00000273874.1,MIR6859-2,...,14.262753,1.415078,15.576286,2.017246,27.847542,3.480999,24.015854,2.438154,19.812614,1.832952
105,105,1,+,629062,629433,ENST00000416931.1,1,372,ENSG00000225972.1,MTND1P23,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
106,106,1,+,629084,630536,MSTRG.3.1,1,1453,MSTRG.3,.,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
# Saving intermediary results
# df_trans_expr.to_csv('BALLGOWN_RES/bg_transcripts_expression.txt', sep='\t', index=False)

In [49]:
ambiguous[0]

['MSTRG.4',
 ['AL627309.1',
  'AL627309.2',
  'AL627309.3',
  'AL627309.5',
  'AL627309.6',
  'AL627309.7',
  'FAM138A',
  'MIR6859-1',
  'MIR6859-2',
  'RNU6-1100P',
  'WASH7P',
  'WASH9P'],
 '-',
 '1',
 14409,
 192299]

#### option B) UCSC MYSQL 

In [55]:
# UCSC mysql connection
conn = pymysql.connect(host='genome-mysql.cse.ucsc.edu',
                       db='hg38',
                       user='genome')

In [56]:
cur = conn.cursor()

In [131]:
query = 'SELECT DISTINCT knownGene.name, kgXref.geneSymbol \
FROM knownGene \
INNER JOIN kgXref ON knownGene.name = kgXref.kgID \
WHERE knownGene.chrom="chr{0}" \
AND knownGene.strand="{1}" \
AND (knownGene.cdsStart BETWEEN "{2}" AND "{3}") \
AND (knownGene.cdsEnd BETWEEN "{4}" AND "{5}") \
ORDER BY knownGene.name;'.format(row["chr"], 
                                 row["strand"], 
                                 row['start'], 
                                 row['end'], 
                                 row['start'], 
                                 row['end'])
query

'SELECT DISTINCT knownGene.name, kgXref.geneSymbol FROM knownGene INNER JOIN kgXref ON knownGene.name = kgXref.kgID WHERE knownGene.chrom="chrY" AND knownGene.strand="+" AND (knownGene.cdsStart BETWEEN "21257840" AND "21291598") AND (knownGene.cdsEnd BETWEEN "21257840" AND "21291598") ORDER BY knownGene.name;'

In [62]:
# TRANSCRIPTS
# Query UCSC mysql server to fill missing gene names 
# search which gene(s) have their CDS starting and ending within the query range
# select the gene name that has the most transcripts assigned to it within the range
# Store list of loci that were not mapped to any name
unknown_list = []
i = 0
j = 0
for index, row in df_trans_expr.iterrows():
    if row['gene_name'] == '.':
        query = 'SELECT DISTINCT knownGene.name, kgXref.geneSymbol \
        FROM knownGene \
        INNER JOIN kgXref ON knownGene.name = kgXref.kgID \
        WHERE knownGene.chrom="chr{0}" \
        AND knownGene.strand="{1}" \
        AND (knownGene.cdsStart BETWEEN "{2}" AND "{3}") \
        AND (knownGene.cdsEnd BETWEEN "{4}" AND "{5}") \
        ORDER BY knownGene.name;'.format(row["chr"], 
                                         row["strand"], 
                                         row['start'], 
                                         row['end'], 
                                         row['start'], 
                                         row['end'])
        cur.execute(query)
        genes_list = []
        for record in cur:
            if record[1] not in genes_list:
                genes_list.append(record[1])
        if len(genes_list) > 0:
            gname = ''
            i+=1
            if len(set(genes_list)) > 1:
                print(colored('MULTI', 'yellow'), end=':  ')
            else:
                print(colored('UNIQUE', 'green'), end=':  ')
            for name in list(set(genes_list)):
                gname = gname + name + ':'
            gname = gname[:-1]
            #print(row["chr"] + ' ' + row["strand"] + ' ' + str(row['start']) + ' ' + str(row['end']) + ' --> ' + gname)
            df_trans_expr.at[index, 'gene_name'] = gname
        else:
            j+=1
            gname = row["chr"] + ':' + row["strand"] + ':' + str(row['start']) + ':' + str(row['end'])
            df_trans_expr.at[index, 'gene_name'] = gname
            #print(colored('UNKNOWN', 'red') + ':  ' + row["chr"] + ':' + row["strand"] + ':' + str(row['start']) + ':' + str(row['end']))
            unknown_list.append([row['gene_id'], row['strand'], row['chr'], row['start'], row['end']])
print(f'Filled {i} gene names. {j} remain unknown, named by coordinates')

UNKNOWN:  1:-:736352:739978
UNKNOWN:  1:-:83444156:83480511
UNKNOWN:  1:-:83444156:83480511
UNKNOWN:  1:-:120844145:120844351
UNKNOWN:  1:+:148529266:148529605
UNKNOWN:  1:-:148661558:148712561
UNKNOWN:  1:+:149636775:149640548
UNKNOWN:  1:+:154999283:154999543
UNKNOWN:  1:+:204506512:204506849
UNKNOWN:  1:+:205957922:205958388
UNKNOWN:  1:+:225999557:226007436
UNKNOWN:  1:-:234610322:234610871
UNKNOWN:  1:-:239623842:239624169
UNKNOWN:  10:+:14185474:14186811
UNKNOWN:  10:+:16891899:16892262
UNKNOWN:  10:-:61903338:61903844
UNKNOWN:  10:-:68632600:68632836
UNKNOWN:  10:+:131751695:131756262
UNKNOWN:  11:-:10509215:10509928
UNKNOWN:  11:+:55898320:55898546
UNKNOWN:  11:-:59550259:59551043
UNKNOWN:  11:-:60072542:60072761
UNKNOWN:  11:-:66183136:66183764
UNKNOWN:  11:-:68459959:68460220
UNKNOWN:  11:+:85483965:85484177
UNKNOWN:  11:-:96346435:96346769
UNKNOWN:  12:-:7723328:7728578
UNKNOWN:  12:+:29833746:29853986
UNKNOWN:  12:-:47218122:47218385
UNKNOWN:  12:-:66003355:66003737
UNKNOWN

UNKNOWN:  GL000219.1:-:84602:99641
UNKNOWN:  GL000220.1:+:109088:159567
UNKNOWN:  GL000220.1:+:109088:159567
UNKNOWN:  GL000220.1:+:109088:159567
UNKNOWN:  GL000220.1:+:109088:159567
UNKNOWN:  GL000220.1:+:109123:159567
UNKNOWN:  GL000220.1:+:109123:159567
UNKNOWN:  GL000220.1:+:109123:161802
UNKNOWN:  GL000220.1:+:109143:159567
UNKNOWN:  GL000220.1:+:109143:159567
UNKNOWN:  GL000220.1:+:109143:159567
UNKNOWN:  GL000220.1:+:113163:118202
UNKNOWN:  GL000220.1:+:113163:118821
UNKNOWN:  GL000220.1:+:113188:118202
UNKNOWN:  GL000220.1:+:113241:117315
UNKNOWN:  GL000220.1:+:113241:117315
UNKNOWN:  GL000220.1:+:113252:118202
UNKNOWN:  GL000220.1:+:113252:118202
UNKNOWN:  GL000220.1:+:113261:118202
UNKNOWN:  GL000220.1:+:113279:118821
UNKNOWN:  GL000220.1:+:113325:118202
UNKNOWN:  GL000220.1:+:113325:118202
UNKNOWN:  GL000220.1:+:113377:118202
UNKNOWN:  GL000220.1:-:113424:115505
UNKNOWN:  GL000220.1:-:113424:116155
UNKNOWN:  GL000220.1:-:113550:154924
UNKNOWN:  GL000220.1:+:114685:118202
UNK

UNKNOWN:  GL000254.2:-:3876401:3891707
UNKNOWN:  GL000254.2:-:3876725:3891707
UNKNOWN:  GL000254.2:-:3876743:3891707
UNKNOWN:  GL000254.2:-:3876743:3891707
UNKNOWN:  GL000254.2:-:3876743:3891707
UNKNOWN:  GL000254.2:-:3876743:3891707
UNKNOWN:  GL000254.2:-:3876743:3891707
UNKNOWN:  GL000254.2:-:3876806:3891536
UNKNOWN:  GL000254.2:-:3877831:3890907
UNKNOWN:  GL000254.2:+:3938453:3952489
UNKNOWN:  GL000254.2:+:3938483:3949863
UNKNOWN:  GL000254.2:-:3952243:3963179
UNKNOWN:  GL000254.2:-:3952730:3963179
UNKNOWN:  GL000254.2:-:3956118:3963179
UNKNOWN:  GL000254.2:+:4045186:4045470
UNKNOWN:  GL000254.2:-:4058403:4058621
UNKNOWN:  GL000254.2:-:4247664:4255178
UNKNOWN:  GL000254.2:-:4303248:4306134
UNKNOWN:  GL000254.2:-:4303248:4308624
UNKNOWN:  GL000254.2:-:4303249:4308624
UNKNOWN:  GL000255.2:+:1189217:1200255
UNKNOWN:  GL000255.2:+:1189265:1204838
UNKNOWN:  GL000255.2:+:1197419:1264295
UNKNOWN:  GL000255.2:+:1197478:1204838
UNKNOWN:  GL000255.2:+:1744602:1751319
UNKNOWN:  GL000255.2:+:19

UNKNOWN:  KI270846.1:+:97517:269551
UNKNOWN:  KI270846.1:-:107339:131944
UNKNOWN:  KI270846.1:-:110870:241410
UNKNOWN:  KI270846.1:-:110879:116157
UNKNOWN:  KI270846.1:-:111004:243619
UNKNOWN:  KI270846.1:-:112272:116094
UNKNOWN:  KI270846.1:+:115396:145572
UNKNOWN:  KI270846.1:-:169155:178562
UNKNOWN:  KI270846.1:-:174857:178562
UNKNOWN:  KI270846.1:-:174873:178562
UNKNOWN:  KI270846.1:-:174884:178985
UNKNOWN:  KI270846.1:-:174884:179240
UNKNOWN:  KI270846.1:-:175554:178562
UNKNOWN:  KI270846.1:-:175562:178985
UNKNOWN:  KI270846.1:-:209086:212507
UNKNOWN:  KI270846.1:-:209097:242795
UNKNOWN:  KI270846.1:-:209097:243619
UNKNOWN:  KI270846.1:-:209097:244236
UNKNOWN:  KI270846.1:-:209104:242795
UNKNOWN:  KI270846.1:-:231894:234891
UNKNOWN:  KI270846.1:-:236788:244236
UNKNOWN:  KI270846.1:-:236809:244236
UNKNOWN:  KI270846.1:-:236842:239993
UNKNOWN:  KI270846.1:-:236871:244236
UNKNOWN:  KI270846.1:-:313054:333175
UNKNOWN:  KI270846.1:-:313144:331582
UNKNOWN:  KI270846.1:-:313144:331983
UN

UNKNOWN:  KZ208911.1:+:236843:242019
UNKNOWN:  KZ208915.1:-:372762:372995
UNKNOWN:  KZ208915.1:-:391879:394308
UNKNOWN:  KZ208915.1:-:410990:411223
UNKNOWN:  KZ208915.1:+:1480382:1505724
UNKNOWN:  KZ208915.1:+:1483044:1505724
UNKNOWN:  KZ208915.1:-:1502940:1545386
UNKNOWN:  KZ208915.1:-:1508680:1545386
UNKNOWN:  KZ208915.1:-:1508817:1545386
UNKNOWN:  KZ208915.1:-:1514756:1545386
UNKNOWN:  KZ208915.1:-:1783483:1854080
UNKNOWN:  KZ208915.1:-:1783483:1854080
UNKNOWN:  KZ208915.1:-:1783516:1854080
UNKNOWN:  KZ208915.1:-:1783516:1854080
UNKNOWN:  KZ208915.1:-:1783517:1854080
UNKNOWN:  KZ208915.1:-:1783517:1854080
UNKNOWN:  KZ208915.1:-:1783582:1854080
UNKNOWN:  KZ208915.1:-:1783669:1854080
UNKNOWN:  KZ208915.1:-:1784364:1854080
UNKNOWN:  KZ208915.1:-:1788731:1854080
UNKNOWN:  KZ208915.1:-:1789800:1854080
UNKNOWN:  KZ208919.1:-:20272:64031
UNKNOWN:  KZ208920.1:+:438144:440263
UNKNOWN:  KZ208920.1:-:517546:543796
UNKNOWN:  KZ208920.1:-:517844:543796
UNKNOWN:  KZ559111.1:-:166431:166649
UNKNOW

In [63]:
cur.close()

In [65]:
unknown_list[500:505]

[['MSTRG.14127', '+', 'KI270733.1', 122287, 179772],
 ['MSTRG.14127', '+', 'KI270733.1', 122591, 125718],
 ['MSTRG.14127', '+', 'KI270733.1', 122594, 128594],
 ['MSTRG.14127', '+', 'KI270733.1', 124065, 128594],
 ['MSTRG.14127', '+', 'KI270733.1', 124070, 179772]]

In [66]:
df_trans_expr.head()

,t_id,chr,strand,start,end,t_name,num_exons,length,gene_id,gene_name,...,cov.BG12,FPKM.BG12,cov.BG13,FPKM.BG13,cov.BG14,FPKM.BG14,cov.BG15,FPKM.BG15,cov.BG16,FPKM.BG16
11,11,1,-,14409,192299,MSTRG.4.8,3,6817,MSTRG.4,AL627309.3:MIR6859-1:AL627309.2:AL627309.6:MIR...,...,14.542374,1.442821,13.637791,1.766196,21.559107,2.694932,20.800241,2.111696,12.477324,1.154332
47,47,1,-,185217,195411,ENST00000623083.4,10,1397,ENSG00000279457.4,WASH9P,...,19.456787,1.930404,26.013676,3.368966,10.763598,1.345472,31.347738,3.182506,0.000000,0.000000
49,49,1,-,187891,187958,ENST00000612080.1,1,68,ENSG00000273874.1,MIR6859-2,...,14.262753,1.415078,15.576286,2.017246,27.847542,3.480999,24.015854,2.438154,19.812614,1.832952
105,105,1,+,629062,629433,ENST00000416931.1,1,372,ENSG00000225972.1,MTND1P23,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
106,106,1,+,629084,630536,MSTRG.3.1,1,1453,MSTRG.3,MTND2P28,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [104]:
len(df_trans_expr)

17306

In [92]:
# Export for further processing in R
df_trans_expr.to_csv('BALLGOWN_RES/bg_transcripts_expression_OK.txt', sep='\t', index=False)

In [68]:
# create dictionary mapping gene_ids to their names from  df_gene_expr for later mapping
i=0
names_dict = {}
for index, row in df_trans_expr.iterrows():
    if row['gene_name'] != '.' and row['gene_id'] not in names_dict:
        i+=1
        names_dict[row['gene_id']] = row['gene_name']
        
print(f'Added {i} gene names to dictionary')

Added 8847 gene names to dictionary


In [105]:
# ALTERNATIVE: add coordinates to dic to fill in datasets
i=0
names_coord_dict = {}
for index, row in df_trans_expr.iterrows():
   # if row['gene_name'] != '.' and row['gene_id'] not in names_coord_dict:
    i+=1
    names_coord_dict[row['gene_id']] = [row['gene_name'], row['chr'], row['strand'], row['start'], row['end']]
print(f'Added {i} gene names to dictionary')

Added 17306 gene names to dictionary


In [106]:
i=0
for k,v in names_coord_dict.items():
    print(k,v)
    i+=1
    if i==3:
        break

MSTRG.4 ['AL627309.3:MIR6859-1:AL627309.2:AL627309.6:MIR6859-2:WASH9P:AL627309.1:AL627309.7:AL627309.5:RNU6-1100P:FAM138A', '1', '-', 14409, 192299]
ENSG00000279457.4 ['WASH9P', '1', '-', 185217, 195411]
ENSG00000273874.1 ['MIR6859-2', '1', '-', 187891, 187958]


### Stringtie - Limma

In [23]:
# import gene_ids list from LIMMA step
df_gene_ids = pd.read_csv('LIMMA_RES/limma_stringtie_geneids.txt', sep='\t', skiprows=(0), header=(0))
df_gene_ids.head()

,ENSEMBL,SYMBOL
0,ENSG00000280143,NaN
1,ENSG00000146083,RNF44
2,ENSG00000269416,LINC01224
3,ENSG00000231981,NaN
4,ENSG00000134108,ARL8B


In [24]:
# how to check if NaN --> NaN != NaN is True
df_gene_ids['SYMBOL'][0] != df_gene_ids['SYMBOL'][0]

True

In [21]:
# Pyensembl function retrieves gene name from ID
data.gene_name_of_gene_id('ENSG00000260500')

'AC010336.2'

In [25]:
# loop over the dataframe and add missing ENSEMBL gene names
i = 0
j = 0
k = 0
for idx, row in df_gene_ids.iterrows():
    if ('ENS' in row['ENSEMBL']) and (row['SYMBOL'] != row['SYMBOL']):
        gname = ''
        gname = data.gene_name_of_gene_id(row['ENSEMBL'])
        if gname != '':
            i+=1
        else:
            j+=1
        df_gene_ids.at[idx, 'SYMBOL'] = gname
    elif 'MSTRG' in row['ENSEMBL']:
        k+=1
print(f'Filled {i} ENSEMBL symbols, still {j} ENSEMBL and {k} MSTRG missing.')

Filled 11228 ENSEMBL symbols, still 0 ENSEMBL and 0 MSTRG missing.


In [26]:
df_gene_ids.to_csv('LIMMA_RES/limma_stringtie_geneids_OK.txt', sep='\t', index=False)

#### Filling missing gene names in RESULTS_GENES from R using names_dic

In [113]:
df_res_genes = pd.read_csv('BALLGOWN_RES/bg_results_genes.txt', sep='\t', skiprows=(0), header=(0))

In [114]:
df_res_genes.head()

,geneNames,feature,id,fc,pval,qval
1,DPM1,gene,ENSG00000000419.12,0.952968,0.861807,0.996319
2,FGR,gene,ENSG00000000938.13,0.751023,0.305573,0.986386
3,GCLC,gene,ENSG00000001084.13,0.986426,0.963085,0.998285
4,NFYA,gene,ENSG00000001167.14,1.268874,0.285631,0.986386
5,LAS1L,gene,ENSG00000001497.16,1.059891,0.790692,0.996319


In [115]:
len(df_res_genes)

8847

In [116]:
# count how many missing gene names
count = 0
for index, row in df_res_genes.iterrows():
    if row['geneNames'] == '.':
        count += 1
print(f'{count} missing gene names')

3871 missing gene names


In [73]:
# filling missing gene names when they exist in names_dict
i=0
j=0
for geneIndex, geneRow in df_res_genes.iterrows():
    if geneRow['geneNames'] == '.' and geneRow['id'] in names_dict:
        i+=1
        df_res_genes.at[geneIndex, 'geneNames'] = names_dict[geneRow['id']]
    elif geneRow['geneNames'] == '.' and geneRow['id'] not in names_dict:
        j+=1
print(f'Filled {i} gene names, still {j} missing')

Filled 3871 gene names, still 0 missing


In [110]:
df_res_genes

,geneNames,feature,id,fc,pval,qval
1,DPM1,gene,ENSG00000000419.12,0.952968,0.861807,0.996319
2,FGR,gene,ENSG00000000938.13,0.751023,0.305573,0.986386
3,GCLC,gene,ENSG00000001084.13,0.986426,0.963085,0.998285
4,NFYA,gene,ENSG00000001167.14,1.268874,0.285631,0.986386
5,LAS1L,gene,ENSG00000001497.16,1.059891,0.790692,0.996319
...,...,...,...,...,...,...
8843,.,gene,MSTRG.9972,0.731863,0.219093,0.986386
8844,.,gene,MSTRG.9979,1.418171,0.463434,0.986386
8845,.,gene,MSTRG.9988,0.803171,0.409197,0.986386
8846,.,gene,MSTRG.9994,1.265494,0.332189,0.986386


In [117]:
# ALTERNATIVE: add coordinates columns
# structure of list: [row['gene_name'], row['chr'], row['strand'], row['start'], row['end']]
i=0
j=0
for geneIndex, geneRow in df_res_genes.iterrows():
    if geneRow['id'] in names_coord_dict:
        i+=1
        df_res_genes.at[geneIndex, 'geneNames'] = names_coord_dict[geneRow['id']][0]
        df_res_genes.at[geneIndex, 'chr'] = names_coord_dict[geneRow['id']][1]
        df_res_genes.at[geneIndex, 'strand'] = names_coord_dict[geneRow['id']][2]
        df_res_genes.at[geneIndex, 'start'] = str(int(names_coord_dict[geneRow['id']][3]))
        df_res_genes.at[geneIndex, 'end'] = str(int(names_coord_dict[geneRow['id']][4]))
    elif geneRow['geneNames'] == '.' and geneRow['id'] not in names_coord_dict:
        j+=1
print(f'Filled {i} gene names, still {j} missing')

Filled 8847 gene names, still 0 missing


In [99]:
len(df_res_genes)

8847

In [63]:
# remove unrecognized genes from dataframe
i = 0
for idx, row in df_res_genes.iterrows():
    if row['geneNames'] == '.':
        i+=1
        df_res_genes.drop(index = idx, inplace=True)
print(f'Removed {i} records missing gene names')

Removed 2068 records missing gene names


In [118]:
df_res_genes

,geneNames,feature,id,fc,pval,qval,chr,strand,start,end
1,DPM1,gene,ENSG00000000419.12,0.952968,0.861807,0.996319,20,-,50934867,50958550
2,FGR,gene,ENSG00000000938.13,0.751023,0.305573,0.986386,1,-,27612669,27626569
3,GCLC,gene,ENSG00000001084.13,0.986426,0.963085,0.998285,6,-,53498756,53544645
4,NFYA,gene,ENSG00000001167.14,1.268874,0.285631,0.986386,6,+,41072983,41097950
5,LAS1L,gene,ENSG00000001497.16,1.059891,0.790692,0.996319,X,-,65512582,65534754
...,...,...,...,...,...,...,...,...,...,...
8843,RUBCN,gene,MSTRG.9972,0.731863,0.219093,0.986386,3,-,197688282,197736907
8844,RPL35A,gene,MSTRG.9979,1.418171,0.463434,0.986386,3,+,197950183,197955899
8845,MIR571:ZNF519P4:ZNF141:AC092574.2,gene,MSTRG.9988,0.803171,0.409197,0.986386,4,+,337814,393561
8846,PCGF3,gene,MSTRG.9994,1.265494,0.332189,0.986386,4,+,705781,770096


In [75]:
len(df_res_genes)

8847

In [119]:
# write final table
df_res_genes.to_csv('BALLGOWN_RES/bg_results_genes_OK.txt', sep='\t', index=False)

#### Filling missing gene names in GENE_EXPRESSION from R using previous names_dict

In [77]:
df_gen_expr = pd.read_csv('BALLGOWN_RES/bg_gene_expression.txt', sep='\t', skiprows=(0), header=(0))

In [78]:
df_gen_expr.head()

,FPKM.BG01,FPKM.BG02,FPKM.BG03,FPKM.BG04,FPKM.BG05,FPKM.BG06,FPKM.BG07,FPKM.BG08,FPKM.BG09,FPKM.BG10,FPKM.BG11,FPKM.BG12,FPKM.BG13,FPKM.BG14,FPKM.BG15,FPKM.BG16,controlMeans,cthMeans,geneNames,geneIDs
1,3.071421,2.308293,0.167408,0.558109,2.590572,3.826903,2.359116,3.349323,1.032477,0.379715,5.431262,1.725221,4.404366,5.050120,0.921382,3.273753,2.278893,2.777287,DPM1,ENSG00000000419.12
2,14.779531,24.619555,6.184224,25.636282,34.901442,20.169939,23.045040,18.480485,39.090947,17.870318,6.338989,3.991157,5.079023,16.229340,15.831924,10.421323,20.977062,14.356628,FGR,ENSG00000000938.13
3,1.140217,1.232682,1.711690,1.948833,0.962728,0.683622,1.750154,4.797488,0.593104,5.229822,0.020015,0.786516,0.000000,2.847026,3.279366,2.021713,1.778427,1.847195,GCLC,ENSG00000001084.13
4,1.679026,1.580273,3.008221,5.118732,0.459134,1.543150,2.100641,1.451065,1.821235,1.569821,1.764375,2.592938,3.985546,4.741767,4.485174,1.161270,2.117530,2.765266,NFYA,ENSG00000001167.14
5,0.932872,1.918391,1.733324,3.588534,0.734940,2.296339,3.210236,2.252500,4.025122,3.611774,3.895302,0.717748,0.678119,1.487848,2.199041,0.956867,2.083392,2.196478,LAS1L,ENSG00000001497.16


In [79]:
i=0
j=0
for geneIndex, geneRow in df_gen_expr.iterrows():
    if geneRow['geneNames'] == '.' and geneRow['geneIDs'] in names_dict:
        i+=1
        df_gen_expr.at[geneIndex, 'geneNames'] = names_dict[geneRow['geneIDs']]
    elif geneRow['geneNames'] == '.' and geneRow['geneIDs'] not in names_dict:
        j+=1
print(f'Filled {i} gene names, still {j} missing')

Filled 3871 gene names, still 0 missing


In [80]:
df_gen_expr.head()

,FPKM.BG01,FPKM.BG02,FPKM.BG03,FPKM.BG04,FPKM.BG05,FPKM.BG06,FPKM.BG07,FPKM.BG08,FPKM.BG09,FPKM.BG10,FPKM.BG11,FPKM.BG12,FPKM.BG13,FPKM.BG14,FPKM.BG15,FPKM.BG16,controlMeans,cthMeans,geneNames,geneIDs
1,3.071421,2.308293,0.167408,0.558109,2.590572,3.826903,2.359116,3.349323,1.032477,0.379715,5.431262,1.725221,4.404366,5.050120,0.921382,3.273753,2.278893,2.777287,DPM1,ENSG00000000419.12
2,14.779531,24.619555,6.184224,25.636282,34.901442,20.169939,23.045040,18.480485,39.090947,17.870318,6.338989,3.991157,5.079023,16.229340,15.831924,10.421323,20.977062,14.356628,FGR,ENSG00000000938.13
3,1.140217,1.232682,1.711690,1.948833,0.962728,0.683622,1.750154,4.797488,0.593104,5.229822,0.020015,0.786516,0.000000,2.847026,3.279366,2.021713,1.778427,1.847195,GCLC,ENSG00000001084.13
4,1.679026,1.580273,3.008221,5.118732,0.459134,1.543150,2.100641,1.451065,1.821235,1.569821,1.764375,2.592938,3.985546,4.741767,4.485174,1.161270,2.117530,2.765266,NFYA,ENSG00000001167.14
5,0.932872,1.918391,1.733324,3.588534,0.734940,2.296339,3.210236,2.252500,4.025122,3.611774,3.895302,0.717748,0.678119,1.487848,2.199041,0.956867,2.083392,2.196478,LAS1L,ENSG00000001497.16


In [71]:
# remove unrecognized genes from dataframe
i = 0
for idx, row in df_gen_expr.iterrows():
    if row['geneNames'] == '.':
        i+=1
        df_gen_expr.drop(index = idx, inplace=True)
print(f'Removed {i} records missing gene names')

Removed 2068 records missing gene names


In [81]:
len(df_gen_expr)

8847

In [83]:
cols = list(df_gen_expr.columns)
cols

['FPKM.BG01',
 'FPKM.BG02',
 'FPKM.BG03',
 'FPKM.BG04',
 'FPKM.BG05',
 'FPKM.BG06',
 'FPKM.BG07',
 'FPKM.BG08',
 'FPKM.BG09',
 'FPKM.BG10',
 'FPKM.BG11',
 'FPKM.BG12',
 'FPKM.BG13',
 'FPKM.BG14',
 'FPKM.BG15',
 'FPKM.BG16',
 'controlMeans',
 'cthMeans',
 'geneNames',
 'geneIDs']

In [84]:
# rearrange columns order
df_gen_expr = df_gen_expr[[cols[-1], cols[-2], cols[0], cols[1], cols[2], cols[3], 
                           cols[4], cols[5], cols[6], cols[7], cols[8], cols[9], 
                           cols[10], cols[11], cols[12], cols[13], cols[14], cols[15], 
                           cols[16], cols[17]]]
df_gen_expr.head()

,geneIDs,geneNames,FPKM.BG01,FPKM.BG02,FPKM.BG03,FPKM.BG04,FPKM.BG05,FPKM.BG06,FPKM.BG07,FPKM.BG08,FPKM.BG09,FPKM.BG10,FPKM.BG11,FPKM.BG12,FPKM.BG13,FPKM.BG14,FPKM.BG15,FPKM.BG16,controlMeans,cthMeans
1,ENSG00000000419.12,DPM1,3.071421,2.308293,0.167408,0.558109,2.590572,3.826903,2.359116,3.349323,1.032477,0.379715,5.431262,1.725221,4.404366,5.050120,0.921382,3.273753,2.278893,2.777287
2,ENSG00000000938.13,FGR,14.779531,24.619555,6.184224,25.636282,34.901442,20.169939,23.045040,18.480485,39.090947,17.870318,6.338989,3.991157,5.079023,16.229340,15.831924,10.421323,20.977062,14.356628
3,ENSG00000001084.13,GCLC,1.140217,1.232682,1.711690,1.948833,0.962728,0.683622,1.750154,4.797488,0.593104,5.229822,0.020015,0.786516,0.000000,2.847026,3.279366,2.021713,1.778427,1.847195
4,ENSG00000001167.14,NFYA,1.679026,1.580273,3.008221,5.118732,0.459134,1.543150,2.100641,1.451065,1.821235,1.569821,1.764375,2.592938,3.985546,4.741767,4.485174,1.161270,2.117530,2.765266
5,ENSG00000001497.16,LAS1L,0.932872,1.918391,1.733324,3.588534,0.734940,2.296339,3.210236,2.252500,4.025122,3.611774,3.895302,0.717748,0.678119,1.487848,2.199041,0.956867,2.083392,2.196478


In [85]:
# write final table
df_gen_expr.to_csv('BALLGOWN_RES/bg_gene_expression_OK.txt', sep='\t', index=False)

#### Filling missing gene names in RESULTS_TRANSCRIPTS from R using names_dict dictionary

In [86]:
df_res_trans = pd.read_csv('BALLGOWN_RES/bg_results_transcripts.txt', sep='\t', skiprows=(0), header=(0))
df_res_trans.head()

,geneNames,geneIDs,feature,id,fc,pval,qval,mean
11,.,MSTRG.4,transcript,11,0.977377,0.916292,0.993393,1.880625
47,WASH9P,ENSG00000279457.4,transcript,47,1.039621,0.905665,0.992615,1.987511
49,MIR6859-2,ENSG00000273874.1,transcript,49,0.880317,0.363163,0.959959,2.670518
105,MTND1P23,ENSG00000225972.1,transcript,105,0.709631,0.271116,0.959959,0.506457
106,.,MSTRG.3,transcript,106,0.804507,0.767515,0.978429,6.057494


In [87]:
# count how many missing gene names
count = 0
for index, row in df_res_trans.iterrows():
    if row['geneNames'] == '.':
        count += 1
print(f'{count} missing gene names')

7995 missing gene names


In [88]:
# filling missing gene names when they exist in names_dict
i=0
j=0
for geneIndex, geneRow in df_res_trans.iterrows():
    if geneRow['geneNames'] == '.' and geneRow['geneIDs'] in names_dict:
        i+=1
        df_res_trans.at[geneIndex, 'geneNames'] = names_dict[geneRow['geneIDs']]
    elif geneRow['geneNames'] == '.' and geneRow['geneIDs'] not in names_dict:
        j+=1
print(f'Filled {i} gene names, still {j} missing')

Filled 7995 gene names, still 0 missing


In [79]:
# remove unrecognized genes from dataframe
i = 0
for idx, row in df_res_trans.iterrows():
    if row['geneNames'] == '.':
        i+=1
        df_res_trans.drop(index = idx, inplace=True)
print(f'Removed {i} records missing gene names')

Removed 4240 records missing gene names


In [89]:
len(df_res_trans)

17306

In [90]:
# write final table
df_res_trans.to_csv('BALLGOWN_RES/bg_results_transcripts_OK.txt', sep='\t', index=False)